In [13]:
import warnings
import functools
import sys
import numpy as np
import types
import numbers

def check_nD(array, ndim, arg_name='image'):
    """
    Verify an array meets the desired ndims and array isn't empty.

    Parameters
    ----------
    array : array-like
        Input array to be validated
    ndim : int or iterable of ints
        Allowable ndim or ndims for the array.
    arg_name : str, optional
        The name of the array in the original function.

    """
    array = np.asanyarray(array)
    msg_incorrect_dim = "The parameter `%s` must be a %s-dimensional array"
    msg_empty_array = "The parameter `%s` cannot be an empty array"
    if isinstance(ndim, int):
        ndim = [ndim]
    if array.size == 0:
        raise ValueError(msg_empty_array % (arg_name))
    if not array.ndim in ndim:
        raise ValueError(msg_incorrect_dim % (arg_name, '-or-'.join([str(n) for n in ndim])))

def _handle_input(image, selem, out, mask, out_dtype=None, pixel_size=1):

    check_nD(image, [2,3])
    if image.dtype not in (np.uint8, np.uint16):
        #message = ('Possible precision loss converting image of type {} to '
        #           'uint8 as required by rank filters. Convert manually using '
        #          'skimage.util.img_as_ubyte to silence this warning.'
        #           .format(image.dtype))
        #warn(message, stacklevel=2)
        #image = img_as_ubyte(image)
        print("warning")

    #selem = np.ascontiguousarray(img_as_ubyte(selem > 0))
    image = np.ascontiguousarray(image)

    if mask is None:
        #Check the its the same dimensions as the regular image
        mask = np.ones(image.shape, dtype=np.uint8)
    else:
        mask = img_as_ubyte(mask)
        mask = np.ascontiguousarray(mask)

    if image is out:
        raise NotImplementedError("Cannot perform rank operation in place.")

    if out is None:
        if out_dtype is None:
            out_dtype = image.dtype
        out = np.empty(image.shape+(pixel_size,), dtype=out_dtype)

        #Check that output dimensions match input dimensions
    else:
        #if len(out.shape) == 2:
        out = out.reshape(out.shape+(pixel_size,))

    is_8bit = image.dtype in (np.uint8, np.int8)

    if is_8bit:
        n_bins = 256
    else:
        # Convert to a Python int to avoid the potential overflow when we add
        # 1 to the maximum of the image.

        #test that .max() works for multiple
        n_bins = int(max(3, image.max())) + 1

    if n_bins > 2**10:
        warn("Bad rank filter performance is expected due to a "
             "large number of bins ({}), equivalent to an approximate "
             "bitdepth of {:.1f}.".format(n_bins, np.log2(n_bins)),
             stacklevel=2)

    return image, selem, out, mask, n_bins

In [14]:
image = np.zeros([100, 100], dtype = np.uint8)

In [15]:
_handle_input(image, None, None, None)

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), None, array([[[ 67],
         [  2],
         [ 91],
         ...,
         [255],
         [255],
         [255]],
 
        [[209],
         [  0],
         [127],
         ...,
         [  1],
         [255],
         [255]],
 
        [[165],
         [  1],
         [170],
         ...,
         [  0],
         [255],
         [255]],
 
        ...,
 
        [[  1],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[136],
         [101],
         [  6],
         ...,
         [167],
         [142],
         [  1]],
 
        [[  1],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]]], dtype=uint8), array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]

In [16]:
x = np.array([[1, 2, 3],[4, 5, 6], [7,8,9]])

In [17]:
x.max()

9

In [18]:
t = np.zeros(x.shape + (1,))

In [19]:
t.shape

(3, 3, 1)

In [49]:
x = np.ones([3,3,3])
print(x)

[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]


In [66]:
t = np.vstack((x, np.zeros((1, x.shape[1], x.shape[2]))))

In [67]:
print(t)

[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [68]:
c = (np.diff(t, axis = 0) < 0).view(np.uint8)

In [69]:
print(c)

[[[0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]]]


In [63]:
from skimage.morphology import disk

In [64]:
print(disk(4))

[[0 0 0 0 1 0 0 0 0]
 [0 0 1 1 1 1 1 0 0]
 [0 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 0 0]
 [0 0 0 0 1 0 0 0 0]]


In [65]:
print(np.ascontiguousarray(c))

[[[0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]]]


In [1]:
from skimage.filters.rank import equalize
import numpy as np

In [2]:
x = np.zeros((10,10,10))
selem = np.ones((3,3,3))

In [3]:
y = equalize(x, selem)

/Users/RyanLu/Documents/scikit-image/skimage/filters/rank/generic.py:179: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  out_dtype)


In [4]:
y.shape

(10, 10, 10)